### Preparing model for scribble app

The main idea for this simple applicatin it is get user's input image descriptor and after that find 3 (or nessesory) with minimum distance from the input image descriptor and descriptors saved in the heap.

For our application we mustn't to train our own net. We chose `resnet` architecture and took one of the trained nets.

* [resnet18](https://download.pytorch.org/models/resnet18-5c106cde.pth)
* [resnet34](https://download.pytorch.org/models/resnet34-333f7ec4.pth)
* [resnet50](https://download.pytorch.org/models/resnet50-19c8e357.pth)
* [resnet101](https://download.pytorch.org/models/resnet101-5d3b4d8f.pth)
* [resnet152](https://download.pytorch.org/models/resnet152-b121ed2d.pth)

In [1]:
import torch
import torch.nn as nn
import torchvision.models as models
from torch.autograd import Variable

In [2]:
from torchvision.models import resnet152

In [11]:
import torch.utils.model_zoo as model_zoo

Chose model architecture and create model object

In [22]:
model = resnet152(pretrained=False, num_classes=1000) # If set the parameter `pretrainded` to True, the constructor is download weights automatically.

In [23]:
device=0

In [24]:
torch.cuda.set_device(device)
model.cuda(device);
#model = torch.nn.DataParallel(model).cuda()

In [25]:
model.load_state_dict(torch.load("resnet152-b121ed2d.pth"))

In [26]:
resnet = list(model.children())

In [50]:
resnet[0]

Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)

In [51]:
layers = resnet

We are copy all layers to the new net, but skip the last layer

In [52]:
model2 = nn.Sequential(*layers[:-1])
for p in model2.parameters():
    p.requires_grad = False

Now we'll do a simple test

In [29]:
import torchvision.transforms as transforms

In [36]:
from PIL import Image
import PIL

In [30]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])

In [83]:
test_transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.Lambda(lambda x: PIL.ImageOps.invert(x)),
            transforms.ToTensor(),
            normalize,
        ])

transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.RandomRotation(15),
            transforms.Lambda(lambda x: PIL.ImageOps.invert(x)),
            transforms.ToTensor(),
            normalize,
        ])

We load one image a few times. Each time the image transformed randomly.

In [96]:
img = Image.open("test2.jpg")

In [97]:
img.show()

In [98]:
img = PIL.ImageOps.invert(img)

In [99]:
img = transform(img)

In [77]:
img = test_transform(img)

In [100]:
input = img.cuda(device)

In [101]:
input.shape

torch.Size([3, 224, 224])

In [102]:
model2.eval()

with torch.no_grad():
    input = input.reshape([1] + list(input.shape))
    input = input.cuda(device, non_blocking=True)
    # inp = inp.cuda(device, non_blocking=True)
    output = model2(input)

In [71]:
v1 = output[0].reshape(-1)

In [81]:
v2 = output[0].reshape(-1)

In [103]:
v22 = output[0].reshape(-1)

The distance between two variats of one randomly rotated image will less than 9 (only for current net architecture and it value found experementally).

In [107]:
torch.dist(v2, v22)

tensor(8.3429, device='cuda:0')

In [106]:
torch.save(model2, 'image224_hash.model')